In [3]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [12]:
def icici2_tab(f):
    try:
        tables=tabula.read_pdf(f,
                                  lattice=True,
                                  pages="all",
                                  silent=True,
                                  multiple_tables=True,
                                  pandas_options={'header':None})
        df = pd.DataFrame()
        df = pd.concat([c for c in tables]).drop_duplicates()
    except: pass
    return df

def icici2_process(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            print("\nICICI Column Headers Missing"); pass
    try:
        df = df.drop(["Init."], axis=1)
    except:
        try:
            df = df.drop(["Branch Name"], axis=1)
        except:pass
        
    try:
        idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
        df.drop(df.index[idx2+1:], inplace=True)
    except:pass
    
    try:
        df = df.loc[:, df.columns.notnull()]
    except:pass
    
    df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance|page total').any(),axis=1)==True].index)]
    df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1, inplace=True)

    df.to_csv("parsed.csv",index=0)
    return df


f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\icici005.pdf'
try:
    df = icici2_tab(f) ; df = icici2_process(df)
    print("Parsed")
except:
    print("Not Parsed")

Parsed


In [29]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\garg_icici\icici04.pdf"
tables=tabula.read_pdf(f,
                          lattice=True,
                          pages="all",
                          silent=True,
                          multiple_tables=True,
                          pandas_options={'header':None})
df = pd.DataFrame()
try:
    df = pd.concat([c for c in tables]).drop_duplicates()
except:pass

df = df.replace(r'^\s*$', np.nan, regex=True)
df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        print("\nICICI Column Headers Missing"); pass
    
try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

try:
    df = df.loc[:, df.columns.notnull()]
except:pass

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df["bal"]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
    except: pass   

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except: pass

try:
    auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
except: pass

try:
    rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
except: pass

for j,i in enumerate(df[rev]):
    if isnan(i) == False:
        df[dep][j] = df[rev][j]
    else:
        pass
    
for j,i in enumerate(df[auto]):
    if isnan(i) == False:
        df[wdl][j] = df[auto][j]
    else:
        pass

df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )

df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )   

df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1

df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float)

df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float)

for i in df.index:
#     # create new column
#     if "Dr" in df["new_column"][i].astype(str):
#         df[bal][i] = df[bal][i]*-1
    if df["bal"][i] == "Dr":
        df[bal][i] = df[bal][i]*-1
    else:
        pass
    
try:
    df.drop(['bal'], axis=1, inplace=True)
except:pass

try:
    df.drop([auto], axis=1, inplace=True)
except:pass

try:
    df.drop([rev], axis=1, inplace=True)
except:pass
    

df.to_csv("check.csv", index=0)
print("parsed")

parsed


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
